In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [3]:
# load data
data_dict = {
    "fleet_path": "data/fleet_small.csv",
    "ports_path": "data/ports_10.csv",
    "dm_path": "data/distance_matrix.csv",
}


ports_df = pd.read_csv(data_dict["ports_path"])
fleet_df = pd.read_csv(data_dict["fleet_path"])
distance_matrix = pd.read_csv(data_dict["dm_path"])

In [8]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel

In [59]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel

In [53]:
envo = CarbonEnv()

In [47]:
envo.action_space_dim[0]

13

In [34]:
envo.embedding_size

128

In [50]:
# something
s = np.array([[0], [0], [0], [1]])


k = np.where(s==0,3*[False],3*[True])

k

k.ravel()

st = tf.convert_to_tensor(s)
st

bm = tf.equal(st,0)


skou = tf.where(bm,tf.repeat(tf.constant(False),3),tf.repeat(tf.constant(True),3))
skou

skou = tf.reshape(skou,[-1])
skou

skou = tf.expand_dims(skou,axis=1)
skou

bol = tf.constant(False,shape=(1,1))

tf.concat((skou,bol),axis=0)

In [66]:
envo.state['ships_state'].shape

AttributeError: 'TensorShape' object has no attribute 'to_list'

In [55]:
initial_state = envo.state

In [56]:
baseline_net = BaselineNet(embedding_size=envo.embedding_size)

In [57]:
policy_net = PolicyNet(embedding_size=envo.embedding_size,output_size=envo.action_space_dim[0])

In [58]:
logits, categorical_dist = policy_net.action_distribution(initial_state)

To shape tou context einai (4, 64)
To shape twn logits prepei na einai 13 kai einai (1, 13)
bhka sto if
to boolean mask tou contracts einai [[ True]
 [False]
 [ True]
 [False]]
to shape tou boolean mask einai (4, 1)
to boolean mask tou actions einai [[False False False]
 [ True  True  True]
 [False False False]
 [ True  True  True]]
ekana flatten thn bm tou actions
to boolean mask tou actions twra einai [False False False  True  True  True False False False  True  True  True]
ebala mia akoma diastash sthn bm tou actions
to boolean mask tou actions twra einai [[False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]]
ebala ena teleutaio true sth bm tou actions
to boolean mask tou actions twra einai [[False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]]
ta logits einai [[ -673.02783 -1256.9597  -5206.2695  -5164.1777   -172.67989  2979.1226
   -310.80475   135.60168 -1139.7727

In [64]:
logits

<tf.Tensor: shape=(4, 13), dtype=float32, numpy=
array([[       -inf,        -inf,        -inf,        -inf,        -inf,
               -inf,        -inf,        -inf,        -inf,        -inf,
               -inf,        -inf,        -inf],
       [       -inf,        -inf,        -inf,        -inf,        -inf,
               -inf,        -inf,        -inf,        -inf,        -inf,
               -inf,        -inf,        -inf],
       [       -inf,        -inf,        -inf,        -inf,        -inf,
               -inf,        -inf,        -inf,        -inf,        -inf,
               -inf,        -inf,        -inf],
       [ -313.3099 ,  1267.9999 ,  -758.8125 ,  1239.5056 , -1898.6394 ,
        -1773.07   , -1779.9456 ,  1001.9906 ,    64.24872, -1404.1523 ,
        -3782.7876 , -1144.2429 , -3108.5317 ]], dtype=float32)>

In [70]:
d = np.reshape(np.arange(1,13),(3,4))
d = tf.convert_to_tensor(d)

In [73]:
d

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])>

In [72]:
tf.math.reduce_mean(d,axis=0)

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([5, 6, 7, 8])>

In [65]:
categorical_dist.sample()

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([13, 13, 13,  1], dtype=int32)>

In [ ]:
initial_state

In [ ]:
value = baseline_net.forward(initial_state)

In [ ]:
baseline_net = BaselineNet()

In [ ]:
envo.observation_space_dim

In [ ]:
envo.action_space_dim

In [ ]:
envo.action_space["actions"].shape

In [ ]:
envo.observation_space["contracts_state"].shape

In [ ]:
envo.observation_space["ships_state"].shape

In [ ]:
envo.observation_space["contracts_mask"].shape

In [ ]:
envo.observation_space["ships_mask"].shape

In [ ]:
ships = envo.create_contracts_tensor()

In [ ]:
ships[0]

In [ ]:
ships[1][:, 4] - 1

In [ ]:
for i in range(0, 8):

    day = i // 4
    print(day)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type

In [ ]:
# Delay

contracts = create_contrs(seed=56)
agent_speed = 10
agent_ship = 2
ship_current_port = 4
contract = contracts.loc[0]
start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_hours = find_distance(ship_current_port,start_port_number) / agent_speed
print(dt_1_hours)
trip_2_hours = find_distance(start_port_number,end_port_number) / agent_speed
print(dt_2_hours)
trip_total_hours = trip_1_hours + trip_2_hours


trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")

# cargo_size vs ship_shize

contracts = create_contrs(seed=56)

# gia ka8e contract se spawned contracts

contract = contracts.loc[0]

fleet_df
# elegxw ka8e ploio pou einai available dhladh exei availability 1 
mask_of_available_ships = fleet_df['ship_availability'] == 1

available_ships = fleet_df[mask_of_available_ships]

available_ships_numbers = available_ships.ship_number

available_ships_numbers



# checkare sto masking an kaneis mask out tous assous h ta mhdenika

# gia ka8e tetoio ploio


# get ship_number

ship_numbers = 
agent_ship_selection = 2

# get ship info
ship_mask = fleet_df['ship_number'] == agent_ship_selection

ship_info = fleet_df[ship_mask]
print(ship_info)

ship_current_port = ship_info.current_port
print(ship_current_port)






start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_distance = find_distance(ship_current_port,start_port_number)
print(trip_1_distance)
trip_2_distance = find_distance(start_port_number,end_port_number)
print(trip_2_distance)
trip_total_distance = trip_1_distance + trip_2_distance




trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")
